In [1]:
import pandas as pd
import numpy as np
import sklearn
from urllib.request import urlopen
from json import loads
# pd.options.display.float_format = '{:.0f}'.format

In [2]:
twenty_eleven_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2011.csv', low_memory=False)
twenty_thirteen_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2013.csv', low_memory=False)
twenty_fifteen_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2015.csv', low_memory=False)

# 2017's data has a lot of missing values and negative values. We don't consider using it at this moment
# twenty_seventeen_df = pd.read_csv('schools_crdc_ap_exams_2017.csv', low_memory=False)

In [3]:
# Read Enrollment Data
twenty_eleven_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2011.csv", low_memory=False)
twenty_thirteen_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2013.csv", low_memory=False)
twenty_fifteen_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2015.csv", low_memory=False)



In [4]:
# missing_id = twenty_eleven_df[twenty_eleven_df['ncessch'].isna()]
# print(len(missing_id) / 30)

# Function to process data that removes all NA rows or with negative values

def data_validifier(dataframe, state_id):
    # Getting rows with the required state_id
    state_df = dataframe[dataframe['fips'] == state_id];
    
    # Aggregate Race and Sex
    state_agg = state_df[(state_df['race'] == 99) & (state_df['sex'] == 99)]
    
    # Drop some columns we don't need at this moment
    state_df_dropcolumn = state_agg.drop(columns=['students_AP_exam_all', 'students_AP_pass_all', 'fips', 'lep', 'disability', 'crdc_id', 
                                                'students_AP_exam_none', 'students_AP_pass_none', 'students_AP_exam_oneormore', 'sex', 'race'])
    
    # Drop NA Values(It's fine to have NA values for exam_all or pass_all
    state_df_nona = state_df_dropcolumn.dropna()
    
    # Remove all rows with negative values
    state_df_final = state_df_nona[(state_df_nona['students_AP_pass_oneormore'] >= 0)]
    
    # Return 2 tables, one with leaid and school id, one without
    school_district_df = state_df_final[['leaid', 'ncessch']]
    state_df_final = state_df_final.drop(columns=['leaid'])
    
    
    return state_df_final, school_district_df

# Get Arizona's valid df
arizona_2011, arizona_2011_district = data_validifier(twenty_eleven_df, 4)
arizona_2013, arizona_2013_district = data_validifier(twenty_thirteen_df, 4)
arizona_2015, arizona_2015_district = data_validifier(twenty_fifteen_df, 4)

print(arizona_2015_district)


           leaid       ncessch
57179   400001.0  4.000010e+10
57749   400019.0  4.000190e+10
58049   400026.0  4.000260e+10
58619   400056.0  4.000560e+10
58799   400065.0  4.000650e+10
...          ...           ...
116129  409630.0  4.096300e+10
116159  409630.0  4.096300e+10
116189  409630.0  4.096300e+10
116219  409630.0  4.096300e+10
116249  409733.0  4.097330e+10

[175 rows x 2 columns]


In [5]:
# Function to process enrollment data
def enrollment_processor(dataframe, state_id):
    # Getting rows with the relevant state_id
    state_enroll_df = dataframe[dataframe['fips'] == state_id]
    
    # Drop preenrollment 
    state_enroll_df = state_enroll_df.drop(columns=['psenrollment_crdc', 'crdc_id', 'fips'])
    
    # We need proportion of females and each races
    # We don't care about disability or LEP here
    state_enroll_relevant = state_enroll_df[(state_enroll_df['disability'] == 99) & (state_enroll_df['lep'] == 99)]
    state_enroll_relevant = state_enroll_relevant.drop(columns=['disability', 'lep'])
    
    # We only want data with ncessch id
    state_enroll_withid = state_enroll_relevant[~state_enroll_relevant['ncessch'].isna()]
    
    # We want proportion and check that with 99
    state_enroll_final = state_enroll_withid[(state_enroll_withid['race'] == 99) | (state_enroll_withid['sex'] == 99)]
    
    return state_enroll_final


arizona_enroll_2011 = enrollment_processor(twenty_eleven_enroll_df, 4)
arizona_enroll_2013 = enrollment_processor(twenty_thirteen_enroll_df, 4)
arizona_enroll_2015 = enrollment_processor(twenty_fifteen_enroll_df, 4)
# We only care about schools that have AP passing one or more in previous table
arizona_enroll_2011 = arizona_enroll_2011[arizona_enroll_2011['ncessch'].isin(arizona_2011['ncessch'])]
arizona_enroll_2013 = arizona_enroll_2013[arizona_enroll_2013['ncessch'].isin(arizona_2013['ncessch'])]
arizona_enroll_2015 = arizona_enroll_2015[arizona_enroll_2015['ncessch'].isin(arizona_2015['ncessch'])]


In [6]:
# Find the percentage of females and different races
def find_percentage(dataframe):
    total_enroll = dataframe[(dataframe['race'] == 99) & (dataframe['sex'] == 99)].copy()
    total_enroll = total_enroll[['ncessch', 'enrollment_crdc']]
    total_enroll.rename(columns={'enrollment_crdc' : 'total_enrollment'}, inplace=True)
    
    merged_total_df = pd.merge(total_enroll, dataframe)
    
    # Find percentage of female and merge
    female_df = merged_total_df[(merged_total_df['sex'] == 2) & (merged_total_df['race'] == 99)].copy()
    female_df['% female'] = female_df['enrollment_crdc'] / female_df['total_enrollment']
    female_df = female_df[['ncessch', '% female']]
    
    
    white_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 1)].copy()
    white_df['% white'] = white_df['enrollment_crdc'] / white_df['total_enrollment']
    white_df = white_df[['ncessch', '% white']]
    
    black_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 2)].copy()
    black_df['% black'] = black_df['enrollment_crdc'] / black_df['total_enrollment']
    black_df = black_df[['ncessch', '% black']]
    
    hispanic_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 3)].copy()
    hispanic_df['% hispanic'] = hispanic_df['enrollment_crdc'] / hispanic_df['total_enrollment']
    hispanic_df = hispanic_df[['ncessch', '% hispanic']]
    
    asian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 4)].copy()
    asian_df['% asian'] = asian_df['enrollment_crdc'] / asian_df['total_enrollment']
    asian_df = asian_df[['ncessch', '% asian']]
    
    american_indian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 5)].copy()
    american_indian_df['% american_indian'] = american_indian_df['enrollment_crdc'] / american_indian_df['total_enrollment']
    american_indian_df = american_indian_df[['ncessch', '% american_indian']]
    
    native_hawaiian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 6)].copy()
    native_hawaiian_df['% native_hawaiian'] = native_hawaiian_df['enrollment_crdc'] / native_hawaiian_df['total_enrollment']
    native_hawaiian_df = native_hawaiian_df[['ncessch', '% native_hawaiian']]
    
    twoormore_races_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 7)].copy()
    twoormore_races_df['% two_or_more_races'] = twoormore_races_df['enrollment_crdc'] / twoormore_races_df['total_enrollment']
    twoormore_races_df = twoormore_races_df[['ncessch', '% two_or_more_races']]
    
    merge_female_df = pd.merge(dataframe, female_df, on='ncessch')
    merge_white_df = pd.merge(merge_female_df, white_df, on='ncessch')
    merge_black_df = pd.merge(merge_white_df, black_df, on='ncessch')
    merge_hispanic_df = pd.merge(merge_black_df, hispanic_df, on='ncessch')
    merge_asian_df = pd.merge(merge_hispanic_df, asian_df, on='ncessch')
    merge_american_indian_df = pd.merge(merge_asian_df, american_indian_df, on='ncessch')
    merge_native_hawaiian_df = pd.merge(merge_american_indian_df, native_hawaiian_df, on='ncessch')
    merge_twoormorerace_df = pd.merge(merge_native_hawaiian_df, twoormore_races_df, on='ncessch')
    final_df = pd.merge(merge_twoormorerace_df, total_enroll, on='ncessch')
    
    final_df = final_df[(final_df['race'] == 99) & (final_df['sex'] == 99)]
    final_df = final_df.drop(columns=['race', 'sex', 'enrollment_crdc', 'leaid'])
    return final_df

def get_info_df(percentage_df, score_df):
    
    info_df = pd.merge(percentage_df, score_df, on='ncessch')
    year_col = info_df['year_y']
    info_df = info_df.drop(columns=['year_x', 'year_y'])
    info_df.insert(1, 'year', year_col)
    return info_df

arizona_2011_enroll_percent = find_percentage(arizona_enroll_2011)
arizona_2013_enroll_percent = find_percentage(arizona_enroll_2013)
arizona_2015_enroll_percent = find_percentage(arizona_enroll_2015)

arizona_2011_info_df = get_info_df(arizona_2011_enroll_percent, arizona_2011)
arizona_2013_info_df = get_info_df(arizona_2013_enroll_percent, arizona_2013)
arizona_2015_info_df = get_info_df(arizona_2015_enroll_percent, arizona_2015)
print(arizona_2011_info_df['ncessch'].nunique())

125


In [7]:
# Concat Data for training purpose
# Planning to use 2011 and 2013 for training
train_data = pd.concat([arizona_2011_info_df, arizona_2013_info_df], ignore_index = True)
train_feature = train_data.drop(columns=['students_AP_pass_oneormore'])
train_target = train_data[['students_AP_pass_oneormore']]

# Test Data is 2015 without passing info
test_data = arizona_2015_info_df
test_feature = test_data.drop(columns=['students_AP_pass_oneormore'])
test_target = test_data[['students_AP_pass_oneormore']]


In [8]:
# Standard Scaling our features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(train_feature)
X_test = scaler.transform(test_feature)


In [9]:
# Use DecisionTreeRegressor to fit the model
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
model = DecisionTreeRegressor(random_state = 482)

# Function to fit and predict different metrics
def predictor():
    
    # Fit and predict students_AP_exam_none
    model.fit(X_train, train_target)
    predict_ap_passing = model.predict(X_test)
    test_result = test_data.copy()
    test_result.rename(columns={'students_AP_pass_oneormore' : 'actual_students_AP_pass_oneormore'}, inplace=True)
    test_result['predicted_students_AP_pass_oneormore'] = predict_ap_passing
    test_result = pd.merge(test_result, arizona_2015_district, on='ncessch')
    return test_result
    
test_result = predictor()

# Calculate the MSE by each district
grouped = test_result.groupby('leaid')
rmse_by_district = {}
mean_residual_by_district = {}

for district_id, group in grouped:
    # rmse = mean_squared_error(group['actual_students_AP_pass_oneormore'], group['predicted_students_AP_pass_oneormore'])
    # rmse_by_district[district_id] = rmse
    
    mean_residual = (group['actual_students_AP_pass_oneormore'] - group['predicted_students_AP_pass_oneormore']).mean()
    mean_residual_by_district[district_id] = mean_residual
    
# for district_id, mse in mse_by_district.items():
#     print(f"District {district_id}: MSE = {mse:.2f}")    
    
for district_id, mean_residual in  mean_residual_by_district.items():
    print(f"District {district_id}: Mean Residual = {mean_residual:.2f}")
    

District 400001.0: Mean Residual = 114.00
District 400019.0: Mean Residual = 10.00
District 400026.0: Mean Residual = 8.00
District 400056.0: Mean Residual = -162.00
District 400065.0: Mean Residual = 0.00
District 400081.0: Mean Residual = 39.00
District 400097.0: Mean Residual = 45.00
District 400112.0: Mean Residual = 32.00
District 400129.0: Mean Residual = 5.00
District 400202.0: Mean Residual = 3.00
District 400211.0: Mean Residual = 4.00
District 400223.0: Mean Residual = 1.00
District 400225.0: Mean Residual = 67.50
District 400327.0: Mean Residual = -9.00
District 400427.0: Mean Residual = -182.00
District 400432.0: Mean Residual = 0.00
District 400450.0: Mean Residual = 10.38
District 400608.0: Mean Residual = 123.00
District 400653.0: Mean Residual = -8.00
District 400680.0: Mean Residual = 55.33
District 400778.0: Mean Residual = -3.00
District 400790.0: Mean Residual = -17.00
District 400818.0: Mean Residual = -67.00
District 400829.0: Mean Residual = -28.00
District 40083

In [19]:
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import count

districts = test_result['leaid']
y_true = test_result['actual_students_AP_pass_oneormore']
y_pred = test_result['predicted_students_AP_pass_oneormore']
regression_metrics = {
    'rmse': root_mean_squared_error,
    'mae': mean_absolute_error,
    'count': count
}

mf = MetricFrame(metrics=regression_metrics, y_true=y_true, y_pred=y_pred, sensitive_features=districts)

print(mf.overall)
print(mf.by_group)
print(mf.difference())
print(mf.ratio())

rmse     118.498262
mae       75.645078
count    193.000000
dtype: float64
                rmse     mae  count
leaid                              
400001.0  114.000000  114.00    1.0
400019.0   10.000000   10.00    1.0
400026.0    8.000000    8.00    1.0
400056.0  162.000000  162.00    1.0
400065.0    0.000000    0.00    1.0
...              ...     ...    ...
408850.0   95.910114   87.25    4.0
409190.0   39.000000   39.00    1.0
409460.0   18.000000   18.00    1.0
409630.0   77.641484   62.60    5.0
409733.0   96.000000   96.00    1.0

[79 rows x 3 columns]
rmse     273.987226
mae      243.588235
count     16.000000
dtype: float64
rmse     0.000000
mae      0.000000
count    0.058824
dtype: float64


In [11]:
# plt.figure(figsize=(100,50), dpi=300)
# plot_tree(model, filled=True, feature_names=train_feature.columns, rounded=True, fontsize=7)
# plt.savefig('APEducationData/decision_tree_plot.png', bbox_inches='tight')  # Save the plot to a file
# plt.close()